In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from sklearn import svm, preprocessing
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

import nltk

from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer

#function to convert lists to strings
def conv_string(x):
    a=[",".join(ingredients) for ingredients in x]
    
    return a

train=pd.read_json('../input/train.json' )
test=pd.read_json('../input/test.json')


#conv_string(train['ingredients'], test['ingredients'])
train['ingredients']=conv_string(train['ingredients'])
test['ingredients']=conv_string(test['ingredients'])


train['ingredients']=[w.lower().split(',') for w in train['ingredients']]
test['ingredients']=[w.lower().split(',') for w in test['ingredients']]

train['ingredients']=conv_string(train['ingredients'])
test['ingredients']=conv_string(test['ingredients'])

tok=sent_tokenize

train['ingredients']=[tok(w) for w in train['ingredients']]
test['ingredients']=[tok(w) for w in test['ingredients']]

train['ingredients']=conv_string(train['ingredients'])
test['ingredients']=conv_string(test['ingredients'])

lemmatizer=WordNetLemmatizer()
train['ingredients']=[lemmatizer.lemmatize(w) for w in train['ingredients']]
test['ingredients']=[lemmatizer.lemmatize(w) for w in test['ingredients']]

print(train['ingredients'].head())
#Use label encoders to convert text labels into integers

lblencdr=preprocessing.LabelEncoder()
y_cuisine=lblencdr.fit_transform(train['cuisine'].values)

#X_train, X_val, y_train, y_val = train_test_split(train['ingredients'], y_cuisine, random_state=0)
vect=TfidfVectorizer(stop_words='english')
X_train_vect=vect.fit_transform(train['ingredients'])
#considering svm model
svmmodel=svm.SVC(kernel='linear', C=1.2)
#mclassmodel=OneVsRestClassifier(svmmodel)
svmmodel.fit(X_train_vect, y_cuisine)

#predictions=svmmodel.predict(vect.transform(test['ingredients']))
#print(f1_score(predictions, train['cuisine'], average='micro'))
test_predict=svmmodel.predict(vect.transform(test['ingredients']))
test_predict_inv=lblencdr.inverse_transform(test_predict)
test['cuisine']=test_predict_inv
test = test.sort_values('id' , ascending=True)

test[['id' , 'cuisine' ]].to_csv("submission.csv", index=False)